In [ ]:
%run "0.0 Data preparation.ipynb"

In [ ]:
filenamePathStem = "stacked bar plots/"

In [ ]:
question0 = shortQuestions['Q01']
game0 = gameDrBugTitle

In [ ]:
question0

In [ ]:
raw20190603.loc[:, [question0, gameQuestion]].loc[raw20190603[gameQuestion]==game0].groupby(question0).size()

In [ ]:
raw20190603.loc[:, [question0, gameQuestion]].head()

In [ ]:
gameQuestion

In [ ]:
raw20190603.loc[:, [question0, gameQuestion]].groupby([gameQuestion, question0]).size()[games[2]]

In [ ]:
data = raw20190603.loc[[question0].value_counts()
data

In [ ]:
games

In [ ]:
def plotStackedBar(question, gameIndex, inputData):
    
    assert(gameIndex < len(games), "game index must be smaller than " + str(len(games)))
    
    game = games[gameIndex]
    data = raw20190603.loc[:, [question0, gameQuestion]].groupby([gameQuestion, question0]).size()
    data = data[game]
    
    N = 1
    counts1 = data["1"]
    counts2 = data["2"]
    counts3 = data["3"]
    counts4 = data["4"]
    counts5 = data["5"]
    ind = np.arange(N)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence

    pg = [[] for i in range(len(games))]
    pg[gameIndex] = [[] for i in range(5)]
    
    pg[gameIndex][0] = plt.bar(ind, counts1, width)
    pg[gameIndex][1] = plt.bar(ind, counts2, width, bottom=counts1)
    pg[gameIndex][2] = plt.bar(ind, counts3, width, bottom=counts1+counts2)
    pg[gameIndex][3] = plt.bar(ind, counts4, width, bottom=counts1+counts2+counts3)
    pg[gameIndex][4] = plt.bar(ind, counts5, width, bottom=counts1+counts2+counts3+counts4)

    plt.ylabel('Counts')
    plt.title(question)
    plt.xticks(ind, games)
    plt.yticks(np.arange(0, data.sum(), round(data.sum()/10)))
    plt.legend((pg[gameIndex][0][0], pg[gameIndex][1][0], pg[gameIndex][2][0], pg[gameIndex][3][0], pg[gameIndex][4][0])
#                , ('1', '2', '3', '4', '5'))
                , data.index.values)

    plt.show()
                           
plotStackedBar(question0, 1, raw20190603)